In [1]:
!pip install transformers datasets sacrebleu sacremoses huggingface_hub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 37.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 KB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 KB 49.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 

In [2]:
from huggingface_hub import notebook_login

notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [3]:
import numpy as np
from datasets import load_dataset, load_metric
from transformers import (AutoTokenizer, AutoModelForSeq2SeqLM, 
                          DataCollatorForSeq2Seq,
                          Seq2SeqTrainingArguments, Seq2SeqTrainer)

In [4]:
raw_datasets = load_dataset("wiki_auto", "manual")
raw_datasets

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/373801 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/73249 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/118074 [00:00<?, ? examples/s]

Dataset wiki_auto downloaded and prepared to /root/.cache/huggingface/datasets/wiki_auto/manual/1.0.0/eeac705719dc9aa2ff180571dfed6c6649588ccdfde8d45a47d2e47e5c5b93af. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['alignment_label', 'normal_sentence_id', 'simple_sentence_id', 'normal_sentence', 'simple_sentence', 'gleu_score'],
        num_rows: 373801
    })
    dev: Dataset({
        features: ['alignment_label', 'normal_sentence_id', 'simple_sentence_id', 'normal_sentence', 'simple_sentence', 'gleu_score'],
        num_rows: 73249
    })
    test: Dataset({
        features: ['alignment_label', 'normal_sentence_id', 'simple_sentence_id', 'normal_sentence', 'simple_sentence', 'gleu_score'],
        num_rows: 118074
    })
})

In [5]:
raw_datasets["train"][0]

{'alignment_label': 1,
 'normal_sentence_id': '0_66252-1-0-0',
 'simple_sentence_id': '0_66252-0-0-0',
 'normal_sentence': 'The Local Government Act 1985 is an Act of Parliament in the United Kingdom.',
 'simple_sentence': 'The Local Government Act 1985 was an Act of Parliament in the United Kingdom.',
 'gleu_score': 0.800000011920929}

In [6]:
max_length = 1024
prefix = "simplify: "

def preproces_function(elements):
    inputs = [prefix + doc for doc in elements["normal_sentence"]]
    outputs = elements["simple_sentence"]
    
    model_inputs = tokenizer(inputs, text_target=outputs, max_length=max_length, truncation=True)

    return model_inputs

In [7]:
model_checkpoint = "t5-small"

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)


tokenized_datasets = raw_datasets.map(preproces_function, batched=True)
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

model_name = model_checkpoint.split("/")[-1]

/usr/local/lib/python3.9/dist-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


Map:   0%|          | 0/373801 [00:00<?, ? examples/s]

Map:   0%|          | 0/73249 [00:00<?, ? examples/s]

Map:   0%|          | 0/118074 [00:00<?, ? examples/s]

In [8]:
num_train_epochs = 1
training_args = Seq2SeqTrainingArguments(
    output_dir=f"{model_name}-finetuned-text-simplification",
    evaluation_strategy="epoch",
    save_strategy = "epoch",
    save_total_limit = 1,
    load_best_model_at_end = True,
    predict_with_generate=True,
    fp16=True,
    include_inputs_for_metrics = True,
    num_train_epochs=num_train_epochs,
    overwrite_output_dir = True,
    push_to_hub=False,
    )

In [9]:
metric = load_metric("sari")

def compute_metrics(eval_pred):
    predictions, labels, inputs = eval_pred

    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    inputs = np.where(inputs != -100, inputs, tokenizer.pad_token_id)

    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    decoded_inputs = tokenizer.batch_decode(inputs, skip_special_tokens=True)
    
    # Some simple post-processing
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]
    decoded_inputs = [[input.strip()] for input in decoded_inputs]

    result = metric.compute(sources=decoded_inputs, predictions=decoded_preds, references=decoded_labels)
    print(result)
    return result

<ipython-input-9-2562c81c081a>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("sari")


In [10]:
trainer = Seq2SeqTrainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["dev"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [11]:
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Sari
1,1.826800,4.418474,58.292136


{'sari': 58.29213592549745}


TrainOutput(global_step=46726, training_loss=2.30136331307512, metrics={'train_runtime': 6589.2437, 'train_samples_per_second': 56.729, 'train_steps_per_second': 7.091, 'total_flos': 6891841114767360.0, 'train_loss': 2.30136331307512, 'epoch': 1.0})

In [27]:
# trainer.save_model("T5_model_and_finetuned_text_simplification")

In [28]:
trainer.save_model("T5_model_and_finetuned_text_simplification")

In [29]:
# After training, access the path of the best checkpoint like this
best_ckpt_path = trainer.state.best_model_checkpoint

In [30]:
best_ckpt_path

't5-small-finetuned-text-simplification/checkpoint-46726'

In [31]:
best_model = AutoModelForSeq2SeqLM.from_pretrained(best_ckpt_path)

In [37]:
# best_model.push_to_hub("T5_model_and_finetuned_text_simplification")

In [33]:
import torch


In [35]:
torch.save(model.state_dict(), 'T5_model_and_finetuned_text_simplification.pt')


In [36]:
from google.colab import files
files.download('T5_model_and_finetuned_text_simplification.pt')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [38]:
from google.colab import drive
drive.mount('/content/drive')
#  To access  GitHub repo and store the model files, you need to first mount your Google Drive in Colab


Mounted at /content/drive


In [52]:
# !git clone https://github.com/HusseinElwakeel/T5_model_and_finetuned_text_simplification.git


In [53]:
# !git remote add origin https://github.com/HusseinElwakeel/T5_model_and_finetuned_text_simplification.git
# !git branch -M main
# !git push -u origin main

In [54]:
# !locale


In [55]:
# !export LC_CTYPE=en_US.UTF-8


In [48]:
# Import the necessary libraries
import os
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Define the path where you want to save the model in your local repo
model_path = "/content/drive/MyDrive/T5_model_and_finetuned_text_simplification/my_model/"

# Create the directory if it doesn't exist
if not os.path.exists(model_path):
    os.makedirs(model_path)

# Save the model and its tokenizer to the local directory
model.save_pretrained(model_path)


In [56]:
# Change directory to the local repository
# !cd /content/T5_model_and_finetuned_text_simplification

# Stage the changes
# !git add .

# Commit the changes with a commit message
# !git commit -m "Add my_model files"

# Push the changes to the remote repository
# !git push origin main
